In [28]:
import pandas as pd
import csv
from matplotlib import pyplot as plt
import os
import numpy as np
from scipy import stats


In [29]:
current_folder = os.getcwd()
csv_list = os.listdir(current_folder)
to_remove = list()
for i in range(len(csv_list)):
    if csv_list[i][-3:] != 'csv':
        to_remove.append(i)
temp = list()
for i in range(len(csv_list)):
    if i not in to_remove:
        temp.append(csv_list[i])
csv_list = temp


In [30]:
I_mm = list()
I_adc = list()
Resistance = list()


In [31]:
for i in csv_list:
    with open(i, "r") as f:
        csv_reader = csv.reader(f, delimiter=',')
        I_adc_temp = list()
        for row in csv_reader:
            # cek kalo list kosong atau engga
            if row:
                I_adc_temp.append(float(row[0]))
    I_adc.append(np.average(I_adc_temp))

    splitted_title = i.replace(".csv", "").split(',')
    I_splitted = splitted_title[0].split(' ')
    I_mm.append(float(I_splitted[0]) +
                    float(I_splitted[1])/(10**len(I_splitted[1])))
    R_temp = splitted_title[1]
    if " " in R_temp:
        R_temp_splitted = R_temp.split(" ")
        before_comma = float(R_temp_splitted[0])
        if  R_temp_splitted[1][-1] == "k":
            before_comma=before_comma*1000
            R_temp_splitted[1]=R_temp_splitted[1].replace("k","")
            after_comma= float(R_temp_splitted[1])*100
        Resistance.append(before_comma+after_comma)
    else:
        if R_temp[-1] == "k":
            R_temp=R_temp.replace("k","")
            Resistance.append(float(R_temp)*1000)
        else:
            Resistance.append(float(R_temp))

In [32]:
Idata = pd.DataFrame([I_mm, I_adc, Resistance]).T
Idata.columns = ['I multimeter', 'I ADC', 'Resistance']


In [33]:
Idata = Idata.sort_values('I multimeter',ascending=False)
#Idata['I ADC'] = Idata['I ADC']*3/6.2/1000


In [35]:
# SIX POINT TWO OHM (6.2 Ohm)
# Gain 2x: 100-150 ohm 
# Gain 4x: 180-220 ohm
# Gain 8x: 330-470 ohm
# Gain 16x: the fucking rest

ADC_Range = [list(range(0, 2)), list(range(2, 4)), list(
    range(4, 6)), list(range(6, len(Idata)))]
ADC_multiplier = [0.0078125, 0.015625, 0.03125, 0.0625][::-1]
multiplier = list()
for i in range(len(ADC_Range)):
    for j in ADC_Range[i]:
        multiplier.append(ADC_multiplier[i])


In [36]:
Idata['I ADC'] = Idata['I ADC'].multiply(multiplier, axis="index")


In [37]:
Idata

,I multimeter,I ADC,Resistance
16,42.9000,1920.421875,100.0
14,30.1200,1237.032500,150.0
13,25.3000,886.457187,180.0
12,21.0200,694.296250,220.0
10,14.3100,495.594375,330.0
9,10.1400,373.264844,470.0
15,4.3900,241.577734,1000.0
11,2.1100,174.234375,2200.0
8,1.4300,152.938672,3300.0
7,1.0150,139.776797,4700.0


In [38]:
Idata.iloc[6:12]

,I multimeter,I ADC,Resistance
15,4.3900,241.577734,1000.0
11,2.1100,174.234375,2200.0
8,1.4300,152.938672,3300.0
7,1.0150,139.776797,4700.0
6,0.7070,140.351562,6800.0
5,0.4836,127.066172,10000.0


In [44]:
tes=stats.linregress(
        Idata['I ADC'], Idata['I multimeter'])

In [47]:
#Idata

,I multimeter,I ADC,Resistance,I interp
16,42.9000,1920.421875,100.0,47.385539
14,30.1200,1237.032500,150.0,29.931678
13,25.3000,886.457187,180.0,20.977935
12,21.0200,694.296250,220.0,16.070117
10,14.3100,495.594375,330.0,10.995244
9,10.1400,373.264844,470.0,7.870930
15,4.3900,241.577734,1000.0,4.507623
11,2.1100,174.234375,2200.0,2.787664
8,1.4300,152.938672,3300.0,2.243769
7,1.0150,139.776797,4700.0,1.907612


In [50]:
regression_range=list.copy(ADC_Range)
regression_range[3]=list(range(6, 12))
Slope_I = list()
Inter_I = list()
for i in regression_range:
    data_I = stats.linregress(
        Idata['I ADC'].iloc[i], Idata['I multimeter'].iloc[i])
    print(data_I)
    Idata['I interp'][i]=Idata['I ADC'][i]*data_I.slope+data_I.intercept
    Slope_I.append(data_I.slope)
    Inter_I.append(data_I.intercept)


LinregressResult(slope=0.018700905322093998, intercept=6.986372337146761, rvalue=0.9999999999999998, pvalue=0.0, stderr=0.0, intercept_stderr=0.0)
LinregressResult(slope=0.022272997080897385, intercept=5.555941650471997, rvalue=1.0, pvalue=0.0, stderr=0.0, intercept_stderr=0.0)
LinregressResult(slope=0.034088252913173835, intercept=-2.5839463973463133, rvalue=1.0, pvalue=0.0, stderr=0.0, intercept_stderr=0.0)
LinregressResult(slope=0.034395938194991385, intercept=-3.9054924417402592, rvalue=0.9968437007347545, pvalue=1.4927615695878522e-05, stderr=0.0013696550996127725, intercept_stderr=0.22883871151717453)


In [54]:
Idata['I ADC']*0.0217

16    41.673155
14    26.843605
13    19.236121
12    15.066229
10    10.754398
9      8.099847
15     5.242237
11     3.780886
8      3.318769
7      3.033156
6      3.045629
5      2.757336
4      2.628399
3      2.534029
2      2.472245
1      2.533022
0      2.505877
Name: I ADC, dtype: float64

In [ ]:
Idata['I interp']=Idata['I ADC']*tes.slope+tes.intercept

In [41]:
data_I

LinregressResult(slope=0.03386681441276144, intercept=-3.803034750173929, rvalue=0.9973991391225308, pvalue=1.3608914505523843e-11, stderr=0.0008157838329336245, intercept_stderr=0.11942447803905358)

In [42]:
for i in range(len(Slope_0)):
    print(Slope_0[i], Inter_0[i])


NameError: name 'Slope_0' is not defined

In [ ]:
res_0 = stats.linregress(Vdata['ADC 0'], Vdata['V multimeter'])
res_1 = stats.linregress(Vdata['ADC 1'], Vdata['V multimeter'])
res_2 = stats.linregress(Vdata['ADC 2'], Vdata['V multimeter'])


In [ ]:
plt.plot(Vdata['ADC 0'], Vdata['V multimeter'], 'o', label='original data')
plt.plot(Vdata['ADC 0'], res_0.intercept + res_0.slope *
         Vdata['ADC 0'], 'r', label='fitted line')


In [ ]:
plt.plot(Vdata['ADC 1'], Vdata['V multimeter'], 'o', label='original data')
plt.plot(Vdata['ADC 1'], res_1.intercept + res_1.slope *
         Vdata['ADC 1'], 'r', label='fitted line')


In [ ]:
plt.plot(Vdata['ADC 2'], Vdata['V multimeter'], 'o', label='original data')
plt.plot(Vdata['ADC 2'], res_2.intercept + res_2.slope *
         Vdata['ADC 2'], 'r', label='fitted line')


In [ ]:
interp_slope = 20.83
